# Seleção dos hyperparâmetros 

Hyperparâmetros são os parâmetros que não são aprendidos no processo de treinamento da rede neural. São parâmetros tanto do modelo, como número e tipo de camadas, 
número de neurônios ou canais em cada camada, como parâmetros do processo de otimização, tais como tipo do otimizador, taxa de aprendizado (learning rate), tamanho
do mini-batch, entre outros.

Uma opção é disparar um processo automático de busca no espaço de hyperparâmetros para buscar o melhor índice de validação cruzada. 

Normalmente a busca neste espaço de hyperparâmetros consiste de:
- o modelo da rede;
- o espaço de hyperparâmetros;
- o método de busca e amostragem neste espaço;
- o esquema de validação cruzada; e
- uma função alvo (*score function*)

Este exemplo utiliza a função do sklearn (scikit-learn) `RandomizedSearchCV` para fazer a busca aleatória no espaço de parâmetros.

<img src='../figures/model_selection.png', width=600></img>

## Validação cruzada

<img src='../figures/cross_validation.png', width=600></img>

## Objetivos desse experimento

Este experimento utiliza como:
- **modelo da rede:** fixa - Fully connected, já usada anteriormente
- **espaço de hyperparâmetros:** variando learning rate e decay
- **método de busca:** RandomizedSearch, onde número de iterações é especificado
- **validação cruzada:** n. de folds especificado, opção se usar ou não dados de teste
- **função alvo:** negativo do loss (tem que ser alvo a ser maximizado)

## Importação dos pacotes tradicionais

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import time
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import torchvision as tv
import lib.pytorch_trainer as ptt

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

np.set_printoptions(precision=3)

GPU available: False


## Importação dos pacotes de seleção de modelo do scikit-learn

In [2]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit
import scipy.stats as st


## Carregamento dos dados - MNIST

In [4]:
train_ds = tv.datasets.MNIST('/data/datasets/MNIST/', train=True, 
                             transform=tv.transforms.ToTensor())
valid_ds = tv.datasets.MNIST('/data/datasets/MNIST/', train=False, 
                             transform=tv.transforms.ToTensor())
n_train = len(train_ds)
n_valid = len(valid_ds)
print('Número de amostras no dataset (treino):', n_train)
print('Número de amostras no dataset (teste ):', n_valid)

Número de amostras no dataset (treino): 60000
Número de amostras no dataset (teste ): 10000


### Reduzindo o tamanho do dataset apenas para acelerar e testar

In [6]:
if True:
    fator_reduc = 0.1
    n_train = int(fator_reduc * n_train)
    n_valid = int(fator_reduc * n_valid)
    train_ds.train_data   = train_ds.train_data[:n_train]
    train_ds.train_labels = train_ds.train_labels[:n_train]
    valid_ds.test_data   = valid_ds.test_data[:n_valid]
    valid_ds.test_labels = valid_ds.test_labels[:n_valid]

In [7]:
print('Número de amostras no dataset (treino):', len(train_ds))
print('Número de amostras no dataset (teste ):', len(valid_ds))

Número de amostras no dataset (treino): 600
Número de amostras no dataset (teste ): 100


### Função de criar um split predefinido utilizando o conjunto de treino concatenado com o conjunto de teste

In [8]:
def torch_datasets_to_sklearn_cv_data(train_ds, valid_ds):
    n_train, n_valid = len(train_ds), len(valid_ds)
    x, y = train_ds[0]
    all_tuples = list(train_ds) + list(valid_ds)
    all_labels = np.array([y for _, y in all_tuples], np.int)
    all_data = torch.cat([w.view(1, *x.shape) for w, _ in all_tuples], 0).numpy()
    valid_fold = np.zeros_like(all_labels)
    valid_fold[:n_train] = -1
    psplit = PredefinedSplit(valid_fold)
    return all_data, all_labels, psplit

### Função para montar o dataset já apropriado para o scikit-learn

In [9]:
def get_dataset(use_test_dataset=False, n_splits=6):
    if use_test_dataset:
        # using the test dataset as a fixed validation set (only one split)
        all_data, all_labels, psplit = torch_datasets_to_sklearn_cv_data(train_ds, valid_ds)

    else:
        all_labels = np.array([y for _, y in list(train_ds)], np.int)
        all_data = torch.cat([w.view(1, 1, 28, 28) for w, _ in list(train_ds)], 0).numpy()
        psplit = n_splits

    # print(all_data.shape, all_data.min(), all_data.max(), '***', all_labels.shape, all_labels.min(), all_labels.max())
    return all_data, all_labels, psplit

### Cria os dados prontos para o scikit-learn usar no `RandomizedSearchCV` e `fit`

In [57]:
use_test_dataset = False
n_splits = 3

all_data, all_labels, psplit = get_dataset(use_test_dataset=use_test_dataset, 
                                           n_splits=n_splits)


In [58]:
print('shape:',all_data.shape, all_labels.shape)
print('psplit:', psplit) # psplit está associado ao número de folders para o cross-validation (CV)

shape: (600, 1, 28, 28) (600,)
psplit: 3


## Definição de rede

In [59]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 50)
        self.at1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.at1(x)
        x = self.fc2(x)
        return x

## Classe para ser chamada pelo `RandomizedSearchCV`

In [60]:
class SklEstimator(BaseEstimator):
    
    skl_id = 0
    fit_num = 0
    
    def __init__(self, model_class=None, criterion_class='CrossEntropyLoss', optim_class='SGD', 
                 optim_lr=0.001, optim_momentum=0.9, weight_decay=0, 
                 sched_step=10, sched_gamma=1.0, mb_size=16, n_epochs=100, verbose=True):
        self.par_model_class = model_class
        self.par_criterion_class = criterion_class
        self.par_optim_class = optim_class
        self.par_optim_lr = optim_lr
        self.par_optim_momentum = optim_momentum
        self.par_weight_decay = weight_decay
        self.par_sched_step = sched_step
        self.par_sched_gamma = sched_gamma
        self.par_mb_size = mb_size
        self.par_n_epochs = n_epochs
        self.par_verbose = verbose
        
    def _initialize(self):
        SklEstimator.skl_id += 1
        self.idd = 'skl_model_{}'.format(SklEstimator.skl_id)
        
        if self.par_model_class is None:
            raise Exception('Model not specified.')
        
        self.model = self.par_model_class()
        
        if self.par_criterion_class == 'CrossEntropyLoss':
            self.criterion = nn.CrossEntropyLoss()
        elif self.par_criterion_class == 'MSELoss':
            self.criterion = nn.MSELoss()
        else:
            self.criterion = None
            raise Exception("A ser implementado ...")
            
        if self.par_optim_class == 'Adam':
            self.optim = torch.optim.Adam(self.model.parameters(), lr=self.par_optim_lr, 
                                          weight_decay=self.par_weight_decay)
        elif self.par_optim_class == 'SGD':
            self.optim = torch.optim.SGD(self.model.parameters(), lr=self.par_optim_lr, 
                                         momentum=self.par_optim_momentum, nesterov=True,
                                         weight_decay=self.par_weight_decay)
        else:
            self.optim = None
            raise Exception("A ser implementado ...")
            
        if self.par_verbose > 0:
            callbacks = [ptt.PrintCallback()]
        else:
            callbacks = None
        
        self.trainer = ptt.DeepNetTrainer(model     = self.model, 
                                          criterion = self.criterion, 
                                          optimizer = self.optim, 
                                          callbacks = callbacks)
    
    def get_params(self, deep):
        params = []
        for k, v in self.__dict__.items():
            if k.startswith('par_'):
                params.append((k[4:], v))
        return dict(params)
    
    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, 'par_' + k, v)
        self._initialize()
        return self
    
    def fit(self, Xtrain, ytrain):
        SklEstimator.fit_num += 1
        self.t0 = time.time()
        print('\n***** Fit #{} '.format(SklEstimator.fit_num))
        Xtra = torch.from_numpy(Xtrain)
        ytra = torch.from_numpy(ytrain)
        self.trainer.fit(self.par_n_epochs, Xtra, ytra, batch_size=self.par_mb_size, shuffle=True)
    
    def score(self, Xtrain, ytrain):
        Xtra = torch.from_numpy(Xtrain)
        ytra = torch.from_numpy(ytrain)
        loss = self.trainer.score(Xtra, ytra, batch_size=self.par_mb_size)
        print('***** Score = {:.5f} [{} samples]  {:.2f}s'.format(loss, ytra.shape[0], time.time() - self.t0))
        return loss

## Espaço de parâmetro para a busca

In [ ]:
parameters_space = {
    'optim_lr':        st.uniform(0.0001, 0.005),
    'weight_decay':    st.uniform(0.0, 0.01),
}

In [ ]:
parameters = {
    'model_class':     [Model],
    'optim_class':     ['Adam'], 
    'mb_size':         [100],
    'n_epochs':        [50],
}

parameters.update(parameters_space) # Junção dos parâmetros fixos e dos a sintonizar

## Inicialização do validator e busca dos melhores parâmetros

In [61]:
n_iterations = 10
validator = RandomizedSearchCV(SklEstimator(verbose=0), 
                               param_distributions = parameters, 
                               cv                  = psplit,
                               n_iter              = n_iterations, 
                               verbose             = 1)

try:
    validator.fit(all_data, all_labels)

except KeyboardInterrupt:
    print('Interrupted!')

## Avaliação dos resultados

### Scores das iterações

In [69]:
print('mean_test_score:',validator.cv_results_['mean_test_score'])
print('rank_test_score:',validator.cv_results_['rank_test_score'])

mean_test_score: [-0.51  -0.518 -0.515 -0.639 -0.519 -0.524 -0.552 -0.521 -0.529 -0.513]
rank_test_score: [ 1  4  3 10  5  7  9  6  8  2]


### Parâmetros utilizados em cada iteração

In [70]:
for par in parameters_space:
    print(par,[ '{:.2}'.format(fit[par]) for fit in validator.cv_results_['params']])

optim_lr ['0.0014', '0.0028', '0.0039', '0.00046', '0.0041', '0.0029', '0.0041', '0.0009', '0.0023', '0.0017']
weight_decay ['0.0077', '0.007', '0.0064', '0.0097', '0.0046', '0.0097', '0.00032', '0.0043', '0.0093', '0.0059']


### Parâmetros de melhor score

In [65]:
validator.best_params_

{'mb_size': 100,
 'model_class': __main__.Model,
 'n_epochs': 50,
 'optim_class': 'Adam',
 'optim_lr': 0.0014482255288768169,
 'weight_decay': 0.0076691268819118101}

In [66]:
validator.best_index_, validator.best_score_

(0, -0.51031715671221412)

In [67]:
validator.best_estimator_.score(all_data, all_labels)

***** Score = -0.10458 [600 samples]  4474.46s


-0.10457577804724376

## Dicionário completo da busca `RandomizedSearchCV`

In [68]:
validator.cv_results_

{'mean_fit_time': array([ 0.619,  0.621,  0.664,  0.663,  0.696,  0.736,  0.639,  0.629,
         0.618,  0.61 ]),
 'mean_score_time': array([ 0.003,  0.003,  0.003,  0.003,  0.005,  0.003,  0.003,  0.003,
         0.003,  0.003]),
 'mean_test_score': array([-0.51 , -0.518, -0.515, -0.639, -0.519, -0.524, -0.552, -0.521,
        -0.529, -0.513]),
 'mean_train_score': array([-0.098, -0.066, -0.056, -0.348, -0.042, -0.085, -0.007, -0.135,
        -0.09 , -0.075]),
 'param_mb_size': masked_array(data = [100 100 100 100 100 100 100 100 100 100],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_model_class': masked_array(data = [<class '__main__.Model'> <class '__main__.Model'> <class '__main__.Model'>
  <class '__main__.Model'> <class '__main__.Model'> <class '__main__.Model'>
  <class '__main__.Model'> <class '__main__.Model'> <class '__main__.Model'>
  <class '__main__.Model'>],
              mask = [False False False Fa

# Exercícios

1. Adicione um parâmetro adicional na escolha do otimizador: 'SGV' e o 'Adam'.
2. Adicione agora um parâmetro na rede, por exemplo trocar a função de ativação de 'ReLU' para 'Sigmoid'.
3. Adicione um parâmetro a sua escolha.